<a href="https://colab.research.google.com/github/sidagarwal04/graph-powered-nlp-workshop/blob/main/graph_powered_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Google's Generative AI, Neo4j Drive and Gradio drivers

In [1]:
!pip install google-generativeai
!pip install -q neo4j-driver
!pip install -q gradio

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.6/381.6 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/6

Import Palm, base64, json, gradio and GraphDatabase library from installed packages/drivers

In [2]:
import google.generativeai as palm
import base64
import json
import gradio as gr
from neo4j import GraphDatabase

Include secret key added as json file

In [3]:
with open ('palm-key.json') as f:
  data = json.load(f)
  palm.configure(api_key = data["API_KEY"])

In [4]:
def get_answer(input):

  defaults = {
    'model': 'models/text-bison-001',
    'temperature': 0.7,
    'candidate_count': 1,
    'top_k': 40,
    'top_p': 0.95,
    'max_output_tokens': 1024,
    'stop_sequences': [],
    'safety_settings': [{"category":"HARM_CATEGORY_DEROGATORY","threshold":1},{"category":"HARM_CATEGORY_TOXICITY","threshold":1},{"category":"HARM_CATEGORY_VIOLENCE","threshold":2},{"category":"HARM_CATEGORY_SEXUAL","threshold":2},{"category":"HARM_CATEGORY_MEDICAL","threshold":2},{"category":"HARM_CATEGORY_DANGEROUS","threshold":2}],
  }

  prompt = f"""You are an expert in converting English questions to Neo4j Cypher Graph code! The Graph has following Node Labels - Movie, Person! the Movie Node has the following properties released, tagline, title. The Person node has properties such as name &amp; born. The Neo4j Graph has the following Relationship types ACTED_IN, DIRECTED, FOLLOWS, PRODUCED, REVIEWED, WROTE!

  All relationships ACTED_IN, DIRECTED, FOLLOWS, PRODUCED, REVIEWED, WROTE start from Person to Movie and not the other way around.

  For example,
  Example 1 - List down 5 movies that released after the year 2000, the Cypher command will be something like this
  ``` MATCH (m:Movie)
  WHERE m.released > 2000
  RETURN m LIMIT 5
  ```

  Example 2 - Get all the people who acted in a movie that was released after 2010.
  ```
  MATCH (p:Person)-[r:ACTED_IN]->(m:Movie)
  WHERE m.released > 2010
  RETURN p,r,m
  ```

  Example 3 - Name the Director of the movie Apollo 13?
  ```
  MATCH (m:Movie)<-[:DIRECTED]-(p:Person)
  WHERE m.title = 'Apollo';
  RETURN p.name
  ```

  Do not include ``` and \n in the output

  {input}"""
  response = palm.generate_text(**defaults,prompt=prompt)

  return response.result

In [5]:
get_answer("Who were the actors in the movie V for Vendetta")

'MATCH (p:Person)-[r:ACTED_IN]->(m:Movie) WHERE m.title = "V for Vendetta" RETURN p.name'

In [6]:
driver = GraphDatabase.driver("neo4j+s://6c9664cc.databases.neo4j.io",
                              auth=("neo4j",
                                    "VTLTI0YR7jUk69YlcT2gAyk_v6UCeLs3_4sz_4iVoWc"))

Import re library for processing regular expressions

In [7]:
import re

Function to remove slash n's (\n) and substituting it with a space in case it exists. For a cleaned query, capture the key-value pair after RETURN expression in cypher query and utilize as a key going-forward

In [8]:
def extract_query_and_return_key(input_query_result):
    slash_n_pattern = r'[ \n]+'
    ret_pattern = r'RETURN\s+(.*)'
    replacement = ' '

    cleaned_query = re.sub(slash_n_pattern, replacement, input_query_result)
    if cleaned_query:
        match = re.search(ret_pattern, cleaned_query)
        if match:
            extracted_string = match.group(1)
        else:
            extracted_string = ""
    return cleaned_query, extracted_string

In [9]:
extract_query_and_return_key(get_answer("Who were the actors in the movie V for Vendetta"))

("MATCH (p:Person)-[r:ACTED_IN]->(m:Movie) WHERE m.title = 'V for Vendetta' RETURN p.name",
 'p.name')

Function to return results as a comma-separated string values with last value having '&' (ampersand/and) in case of list of values.

In [10]:
def format_names_with_ampersand(names):
    if len(names) == 0:
        return ""
    elif len(names) == 1:
        return names[0]
    else:
        formatted_names = ", ".join(names[:-1]) + " & " + names[-1]
        return formatted_names

In [11]:
format_names_with_ampersand(["Sachin","Virat","Rahul"])

'Sachin, Virat & Rahul'

Run Cypher on Neo4j function

In [29]:
def run_cypher_on_neo4j(inp_query, inp_key):
    out_list = []
    with driver.session() as session:
        result = session.run(inp_query)
        for record in result:
            out_list.append(record[inp_key])
    driver.close()
    if len(out_list) > 1:
        return format_names_with_ampersand(out_list)
    elif len(out_list) == 1:
        return out_list[0]
    else:
        return ""

In [30]:
query_1, string_1 = extract_query_and_return_key(get_answer("Who were the actors in the movie Cast Away"))

run_cypher_on_neo4j (query_1, string_1)

<ipython-input-29-79aa26888c25>:3: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


'Helen Hunt & Tom Hanks'

In [14]:
def generate_and_exec_cypher(input_query):
    gen_query, gen_key = extract_query_and_return_key(get_answer(input_query))
    return run_cypher_on_neo4j(gen_query, gen_key)

In [ ]:
def chatbot(input, history=[]):
    output = str(generate_and_exec_cypher(input))
    history.append((input, output))
    return history, history

In [ ]:
gr.Interface(fn = chatbot,
             inputs = ["text",'state'],
             outputs = ["chatbot",'state']).launch(debug = True)